# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify_db
        WITH REPLICATION = {'class': 'SimpleStrategy', 
        'replication_factor': 1}
    """)
except Exception as error:
    print(error)

#### Set Keyspace

In [7]:

session.set_keyspace('sparkify_db')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
#===============================================================================================
# For table songinfo_by_SessionAndItem, the sessionID was 
# used as a partition key because the queries use it to filter. The itemInSession were used as 
# clustering column to generate unique key.
#==============================================================================================


try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_by_SessionAndItem(
        sessionID int, 
        itemInSession int, 
        artist text,
        song text, 
        length float, 
        PRIMARY KEY (sessionID, itemInSession)
        );
    """)
except Exception as error:
    print(error)
                    

In [10]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songinfo_by_SessionAndItem (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:

result = session.execute("""
    SELECT artist, song, length
    FROM songinfo_by_SessionAndItem
    WHERE
        sessionId=338 AND
        itemInSession=4;
""")

print(result._current_rows)


[Row(sessionid=338, iteminsession=4, artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')]


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
#====================================================================================
# For table songinfo_byUserAndSession, userId and 
# sessionId were used as a composite partition key 
# because the queries use it as a filter. 
# The itemInSession functions as the clustering column to make a unique primary key
#====================================================================================



try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_byUserAndSession(
            userID int, 
            sessionID int, 
            itemInSession int, 
            artist text, 
            song text, 
            firstName text, 
            lastName text, 
            PRIMARY KEY ((userID, sessionID), itemInSession)
        );
    """)

except Exception as error:
    print(error)
                    

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_byUserAndSession (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [20]:
res = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM songinfo_byUserAndSession
    WHERE
        userId=10 AND
        sessionId=182
    ORDER BY itemInSession;
""")
for elem in res._current_rows:
    print(elem)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [21]:
#========================================================================
# For table users_by_song, the song was used as a 
# partition key as it is used as the only filter when performing queries.
# userId was used as a clustering column to make the primary key unique.
#========================================================================

try: 
    session.execute("""
        CREATE TABLE IF NOT EXISTS users_by_song(
            song text, 
            firstName text, 
            lastName text, 
            userID int, 
            PRIMARY KEY ((song), userID)
        );
    """)
except Exception as error:
    print(error)
                    

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [24]:
res = session.execute("""
    SELECT firstName, lastName
    FROM users_by_song
    WHERE song='All Hands Against His Own';
""")

for elem in res._current_rows:
    print(elem)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [25]:
try:
    session.execute("DROP TABLE songinfo_by_SessionAndItem")
    session.execute("DROP TABLE songinfo_byUserAndSession")
    session.execute("DROP TABLE users_by_song")
except Exception as err:
    print(err)

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()